In [132]:
# Re-run this cell 
import pandas as pd

In [133]:
# Read in the data
schools = pd.read_csv("schools.csv")

schools.head()

,school_name,borough,building_code,average_math,average_reading,average_writing,percent_tested
0,"New Explorations into Science, Technology and ...",Manhattan,M022,657,601,601,NaN
1,Essex Street Academy,Manhattan,M445,395,411,387,78.9
2,Lower Manhattan Arts Academy,Manhattan,M445,418,428,415,65.1
3,High School for Dual Language and Asian Studies,Manhattan,M445,613,453,463,95.9
4,Henry Street School for International Studies,Manhattan,M056,410,406,381,59.7


## EDA

In [134]:
schools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   school_name      375 non-null    object 
 1   borough          375 non-null    object 
 2   building_code    375 non-null    object 
 3   average_math     375 non-null    int64  
 4   average_reading  375 non-null    int64  
 5   average_writing  375 non-null    int64  
 6   percent_tested   355 non-null    float64
dtypes: float64(1), int64(3), object(3)
memory usage: 20.6+ KB


* 375 observations
* 6 feature columns

### Categorical features

In [135]:
cat_feat = schools.select_dtypes(include=object)
cat_feat.head()

,school_name,borough,building_code
0,"New Explorations into Science, Technology and ...",Manhattan,M022
1,Essex Street Academy,Manhattan,M445
2,Lower Manhattan Arts Academy,Manhattan,M445
3,High School for Dual Language and Asian Studies,Manhattan,M445
4,Henry Street School for International Studies,Manhattan,M056


In [136]:
for col in cat_feat: 
    print(cat_feat[col].value_counts(), '\n')

New Explorations into Science, Technology and Math High School    1
Multicultural High School                                         1
High School for Enterprise, Business, and Technology              1
PROGRESS High School for Professional Careers                     1
Midwood High School                                               1
                                                                 ..
Urban Assembly School for Applied Math and Science                1
Frederick Douglass Academy III Secondary School                   1
Morris Academy for Collaborative Studies                          1
School for Excellence                                             1
Scholars' Academy                                                 1
Name: school_name, Length: 375, dtype: int64 

Brooklyn         109
Bronx             98
Manhattan         89
Queens            69
Staten Island     10
Name: borough, dtype: int64 

X425    6
X435    6
X415    5
M490    5
M535    5
       ..
X973    1


### Numerical features

In [137]:
num_feat = schools.select_dtypes(exclude=object)
num_feat.head()

,average_math,average_reading,average_writing,percent_tested
0,657,601,601,NaN
1,395,411,387,78.9
2,418,428,415,65.1
3,613,453,463,95.9
4,410,406,381,59.7


In [138]:
num_feat.describe()

,average_math,average_reading,average_writing,percent_tested
count,375.000000,375.000000,375.000000,355.000000
mean,432.944000,424.504000,418.458667,64.976338
std,71.952373,61.881069,64.548599,18.747634
min,317.000000,302.000000,284.000000,18.500000
25%,386.000000,386.000000,382.000000,50.950000
50%,415.000000,413.000000,403.000000,64.800000
75%,458.500000,445.000000,437.500000,79.600000
max,754.000000,697.000000,693.000000,100.000000


### NUll values

In [139]:
schools.isnull().sum()

school_name         0
borough             0
building_code       0
average_math        0
average_reading     0
average_writing     0
percent_tested     20
dtype: int64

* Percent tested is the only feature with null values

## Summary Statistics

* Wich NYC schools have the best math results?

In [140]:
schools.columns

Index(['school_name', 'borough', 'building_code', 'average_math',
       'average_reading', 'average_writing', 'percent_tested'],
      dtype='object')

In [141]:
cols = ['school_name', 'average_math']
threshold_score = 0.8 * 800
mask_best_math_schools = schools['average_math'] >= threshold_score

In [142]:
best_math_schools = schools[mask_best_math_schools][cols].sort_values(by='average_math', ascending=False)
best_math_schools.head()

,school_name,average_math
88,Stuyvesant High School,754
170,Bronx High School of Science,714
93,Staten Island Technical High School,711
365,Queens High School for the Sciences at York Co...,701
68,"High School for Mathematics, Science, and Engi...",683


* What are the top 10 performing schools based on the combined SAT scores?

In [143]:
schools['total_SAT'] = schools['average_math'] + schools['average_reading'] + schools['average_writing']
schools.head()

,school_name,borough,building_code,average_math,average_reading,average_writing,percent_tested,total_SAT
0,"New Explorations into Science, Technology and ...",Manhattan,M022,657,601,601,NaN,1859
1,Essex Street Academy,Manhattan,M445,395,411,387,78.9,1193
2,Lower Manhattan Arts Academy,Manhattan,M445,418,428,415,65.1,1261
3,High School for Dual Language and Asian Studies,Manhattan,M445,613,453,463,95.9,1529
4,Henry Street School for International Studies,Manhattan,M056,410,406,381,59.7,1197


In [144]:
cols2 = ['school_name', 'total_SAT']

In [145]:
top_10_schools = schools.sort_values(by='total_SAT', ascending=False)[cols2].head(10)
top_10_schools

,school_name,total_SAT
88,Stuyvesant High School,2144
170,Bronx High School of Science,2041
93,Staten Island Technical High School,2041
174,High School of American Studies at Lehman College,2013
333,Townsend Harris High School,1981
365,Queens High School for the Sciences at York Co...,1947
5,Bard High School Early College,1914
280,Brooklyn Technical High School,1896
45,Eleanor Roosevelt High School,1889
68,"High School for Mathematics, Science, and Engi...",1889


* Which single borough has the largest standard deviation in the combined SAT score?

In [146]:
schools_by_borough = schools.groupby('borough')['total_SAT'].agg(['count', 'mean', 'std']).round(2)
schools_by_borough.rename(
        columns={'count': 'num_schools', 'mean': 'average_SAT', 'std': 'std_SAT'}, 
        inplace=True
    )
schools_by_borough

,num_schools,average_SAT,std_SAT
borough,,,
Bronx,98,1202.72,150.39
Brooklyn,109,1230.26,154.87
Manhattan,89,1340.13,230.29
Queens,69,1345.48,195.25
Staten Island,10,1439.00,222.30


In [147]:
largest_std_dev = schools_by_borough[
    schools_by_borough['std_SAT'] == schools_by_borough['std_SAT'].max()
]
largest_std_dev

,num_schools,average_SAT,std_SAT
borough,,,
Manhattan,89,1340.13,230.29


## Plotting results